# EmpkinS Micro – ECG Data Preprocessing

In [3]:
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp

from tqdm.notebook import tqdm

from empkins_micro.time_logs import load_split_time_logs

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

[(0.0, 0.2196078431372549, 0.396078431372549),
 (0.596078431372549, 0.6431372549019608, 0.6823529411764706),
 (0.788235294117647, 0.5764705882352941, 0.07450980392156863),
 (0.0, 0.6941176470588235, 0.9215686274509803),
 (0.0, 0.6078431372549019, 0.4666666666666667),
 (0.5529411764705883, 0.0784313725490196, 0.1607843137254902)]

In [13]:
base_path = Path("../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy")
data_path = base_path.joinpath("data_per_subject")

subject_dirs = bp.utils.file_handling.get_subject_dirs(data_path, "VP_*")

In [14]:
subject_dirs

[PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_01'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_02'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_03'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_04'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_05'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_06'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_07'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_08'),
 PosixPath('../../../../../../HealthPsychology_D03/Data/2021_06_Micro_Prestudy/data_per_subject/VP_09')]

In [15]:
subject_dict = {}

for subject_dir in tqdm(subject_dirs[:1], desc="Subject"):
    subject_id = subject_dir.name
    
    for condition in tqdm(["control", "stress"], desc=subject_id):        
        ecg_dir = subject_dir.joinpath("{}/ecg").format(condition)
        timelog_dir = subject_dir.joinpath("{}/time_log").format(condition))
        timelog_dict = load_split_time_logs(timelog_dir.joinpath("processed"))
        
        ecg_dict, fs = bp.io.nilspod.load_folder_nilspod(ecg_dir.joinpath("raw"), phase_names=["Pre", "MIST", "Post"])
        
        ep_dict = {}
        for key_timelog, key_ecg in tqdm(list(zip(timelog_dict, ecg_dict)), desc=condition):
            ep = bp.signals.ecg.EcgProcessor(data=ecg_dict[key_ecg], sampling_rate=fs, time_intervals=timelog_dict[key_timelog])
            ep.ecg_process(title=key_ecg)
            
            ep_dict[key_ecg] = ep

            ecg_export_dir = ecg_dir.joinpath("processed")
            ecg_export_dir.mkdir(exist_ok=True)
            bp.io.write_pandas_dict_excel(ep.heart_rate, ecg_export_dir.joinpath("heart_rate_result_{}_{}.xlsx".format(subject_id, key_timelog)))
            bp.io.write_pandas_dict_excel(ep.rpeaks, ecg_export_dir.joinpath("rpeaks_result_{}_{}.xlsx".format(subject_id, key_timelog)))
            
        subject_dict[subject_id] = ep_dict


Subject:   0%|          | 0/1 [00:00<?, ?it/s]

VP_01:   0%|          | 0/2 [00:00<?, ?it/s]

control:   0%|          | 0/3 [00:00<?, ?it/s]

Pre:   0%|          | 0/2 [00:00<?, ?it/s]

MIST:   0%|          | 0/9 [00:00<?, ?it/s]

Post:   0%|          | 0/1 [00:00<?, ?it/s]

stress:   0%|          | 0/3 [00:00<?, ?it/s]

Pre:   0%|          | 0/2 [00:00<?, ?it/s]

MIST:   0%|          | 0/9 [00:00<?, ?it/s]

Post:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
bp.signals.ecg.plotting.ecg_plot(ep, key="post")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 1000x500 with 4 Axes>,
 [<AxesSubplot:ylabel='ECG Signal (z-norm.)'>,
  <AxesSubplot:xlabel='Time', ylabel='Heart Rate [bpm]'>,
  <AxesSubplot:title={'center':'Individual Heart Beats'}>,
  <AxesSubplot:title={'center':'Heart Rate Distribution'}, xlabel='Heart Rate [bpm]'>])